In [4]:
import pandas as pd

In [4]:
from dataclasses import dataclass


@dataclass(frozen=True)
class Observation:
    hr: int
    bp: int
    temp: float
    resp: float
    ox_sats: float
    insp_ox: float

In [5]:
obs_1 = Observation(hr=70, bp=115, temp=37, resp=36.6, ox_sats=98, insp_ox=0)

<h1> Membership Functions

In [6]:
# take in an input int -> determine how much it belongs to the mf -> output that value

class tri_mf:
    def __init__(self, x, a, b, c):
        self.x = x
        self.a = a
        self.b = b
        self.c = c
        
    def __repr__(self):
        return f'Lower_Bound = {self.a}, Centre = {self.b}, Upper_Bound = {self.c}'

    def __call__(self, x, a, b, c):
        if x <= a or x >= c:
            return 0
        elif x > a and x < b:
            return (x - a) / (b - a)
        elif x == b:
            return 1.0
        elif x > b and x < c:
            return (c - x) / (c - b)
        
        
class trap_mf:
    def __init__(self, a, b, c, d):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        
    def __repr__(self):
        return f'Lower_bound: {self.a}, Centre_point_1: {self.b}, Centre_point_2: {self.c}, Upper_bound: {self.d}'
    
    def __call__(self, x):
        if x <= self.a or x >=self.d:
            return 0.0
        elif x > self.a and x < self.b:
            return (x - self.a) / (self.b - self.a)
        elif x > self.b and x < self.c or x == self.b or self.c:
            return 1.0
        elif x > self.c and x < self.d:
            return (self.d - x) / (self.d - self.c)
        

<h1> Fuzzy Variables

In [7]:
class FuzzyVariable:
    def __init__(self, name, mfs: dict):
        self.name = name
        self.mfs = mfs
        
    def fuzzyify(self, x):
        return {label: mf(x) for label, mf in self.mfs.items()}

In [8]:
heart_rate = FuzzyVariable(
    name = "Heart Rate",
    mfs ={
    "Low": trap_mf(0, 40, 50, 60),
    "Normal" : trap_mf(50,60,80,100),
    "High" : trap_mf(90,110,130,160),
    "Very High": trap_mf(120,140,200,220)
    })

print(heart_rate.fuzzyify(110))

{'Low': 0.0, 'Normal': 0.0, 'High': 1.0, 'Very High': 0.0}


In [218]:
class custom_mf_7_var:
    def __init__(self, path):
        self.path = path
        self.df = pd.read_csv(path)
        
        keys = self.df.loc[:, 'Value'].values
        
        self.B_SevC = dict(zip(keys, self.df.loc[:, 'Below normal - severe concern'].values))
        self.B_ModC = dict(zip(keys, self.df.loc[:, 'Below normal - moderate concern'].values))
        self.B_MildC = dict(zip(keys, self.df.loc[:, 'Below normal - mild concern'].values))
        self.no_con = dict(zip(keys, self.df.loc[:,'No concern'].values))
        self.A_MildC = dict(zip(keys, self.df.loc[:, 'Above normal - mild concern'].values))
        self.A_ModC = dict(zip(keys, self.df.loc[:, 'Above normal - moderate concern'].values))
        self.A_SevC = dict(zip(keys, self.df.loc[:, 'Above normal - severe concern'].values))
        
        self.fs = [self.B_SevC, self.B_ModC, self.B_MildC, self.no_con, self.A_MildC, self.A_ModC, self.A_SevC]
        self.labels = ['Below normal - severe concern', 'Below normal - moderate concern', 
                       'Below normal - mild concern', 'No concern', 
                       'Above normal - mild concern', 'Above normal - moderate concern', 
                       'Above normal - severe concern']
    
    def __call__(self, inp):
        out = {}
        for label, fs in zip(self.labels, self.fs):
            membership_value = fs.get(inp, 0.0)
            out[label] = membership_value
        
        return out
    
class custom_mf_3_var_up:
    def __init__(self, path):
        self.path = path
        self.df = pd.read_csv(path)
        
        keys = self.df.loc[:, 'Value'].values
        
        self.no_con = dict(zip(keys, self.df.loc[:,'No concern'].values))
        self.A_MildC = dict(zip(keys, self.df.loc[:, 'Above normal - mild concern'].values))
        self.A_ModC = dict(zip(keys, self.df.loc[:, 'Above normal - moderate concern'].values))
        self.A_SevC = dict(zip(keys, self.df.loc[:, 'Above normal - severe concern'].values))
        
        self.fs = [self.no_con, self.A_MildC, self.A_ModC, self.A_SevC]
        self.labels = ['No concern', 
                       'Above normal - mild concern', 'Above normal - moderate concern', 
                       'Above normal - severe concern']
    
    def __call__(self, inp):
        out = {}
        for label, fs in zip(self.labels, self.fs):
            membership_value = fs.get(inp, 0.0)
            out[label] = membership_value
        
        return out

        
class custom_mf_3_var_down:
    def __init__(self, path):
        self.path = path
        self.df = pd.read_csv(path)
        
        keys = self.df.loc[:, 'Value'].values
        
        self.B_SevC = dict(zip(keys, self.df.loc[:, 'Below normal - severe concern'].values))
        self.B_ModC = dict(zip(keys, self.df.loc[:, 'Below normal - moderate concern'].values))
        self.B_MildC = dict(zip(keys, self.df.loc[:, 'Below normal - mild concern'].values))
        self.no_con = dict(zip(keys, self.df.loc[:,'No concern'].values))

        self.fs = [self.B_SevC, self.B_ModC, self.B_MildC, self.no_con]
        self.labels = ['Below normal - severe concern', 'Below normal - moderate concern', 
                       'Below normal - mild concern', 'No concern']
    
    def __call__(self, inp):
        out = {}
        for label, fs in zip(self.labels, self.fs):
            membership_value = fs.get(inp, 0.0)
            out[label] = membership_value
        
        return out

In [219]:
heart_rate = custom_mf_7_var('data/membership_function_plots/csv_data/heart_rate_membership_functions.csv')
blood_pressure = custom_mf_7_var('data/membership_function_plots/csv_data/systolic_blood_pressure_membership_functions.csv')
temperature = custom_mf_7_var('data/membership_function_plots/csv_data/temperature_membership_functions.csv')
respiratory_rate = custom_mf_7_var('data/membership_function_plots/csv_data/respiratory_rate_membership_functions.csv')
oxygen_saturation = custom_mf_3_var_down('data/membership_function_plots/csv_data/oxygen_saturation_membership_functions.csv')
inspired_oxygen = custom_mf_3_var_up('/home/msxtk13/Documents/FuzzySystemMain/data/membership_function_plots/csv_data/inspired_oxygen_concentration_membership_functions.csv')


print(inspired_oxygen(80))
print(heart_rate(90))
print(blood_pressure(120))
print(temperature(37))
print(respiratory_rate(18))
print(oxygen_saturation(95))


{'No concern': np.float64(0.0), 'Above normal - mild concern': np.float64(0.0), 'Above normal - moderate concern': np.float64(0.0050761421319796), 'Above normal - severe concern': np.float64(0.9949238578680204)}
{'Below normal - severe concern': np.float64(0.0), 'Below normal - moderate concern': np.float64(0.0), 'Below normal - mild concern': np.float64(0.0), 'No concern': np.float64(1.0), 'Above normal - mild concern': np.float64(0.0), 'Above normal - moderate concern': np.float64(0.0), 'Above normal - severe concern': np.float64(0.0)}
{'Below normal - severe concern': np.float64(0.0), 'Below normal - moderate concern': np.float64(0.0), 'Below normal - mild concern': np.float64(0.0), 'No concern': np.float64(1.0), 'Above normal - mild concern': np.float64(0.0), 'Above normal - moderate concern': np.float64(0.0), 'Above normal - severe concern': np.float64(0.0)}
{'Below normal - severe concern': np.float64(0.0), 'Below normal - moderate concern': np.float64(0.0), 'Below normal - mild 

In [154]:
print(heart_rate(90))


(0       20
1       21
2       22
3       23
4       24
      ... 
176    196
177    197
178    198
179    199
180    200
Name: Value, Length: 181, dtype: int64, 0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
176    0.0
177    0.0
178    0.0
179    0.0
180    0.0
Name: Below normal - severe concern, Length: 181, dtype: float64)
(0       20
1       21
2       22
3       23
4       24
      ... 
176    196
177    197
178    198
179    199
180    200
Name: Value, Length: 181, dtype: int64, 0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
176    0.0
177    0.0
178    0.0
179    0.0
180    0.0
Name: Below normal - moderate concern, Length: 181, dtype: float64)
(0       20
1       21
2       22
3       23
4       24
      ... 
176    196
177    197
178    198
179    199
180    200
Name: Value, Length: 181, dtype: int64, 0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
176    0.0
177    0.0
178    0.0
179    0.0
180    0.0
Name: Below norma